In [1]:
import pandas as pd
data=pd.read_csv("C:/Users/Rajesh/Documents/me/data_assignments/mod16/book.csv", encoding='latin-1')
data.head()

,Unnamed: 0,users_id,Book.Title,Book.Author,Publisher,"ratings[, 3]"
0,1,1,Classical Mythology,Mark P. O. Morford,Oxford University Press,0
1,2,2,Clara Callan,Richard Bruce Wright,HarperFlamingo Canada,5
2,3,3,Decision in Normandy,Carlo D'Este,HarperPerennial,0
3,4,4,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,Farrar Straus Giroux,3
4,5,5,The Mummies of Urumchi,E. J. W. Barber,W. W. Norton &amp; Company,6


In [2]:
#EDA
data.shape
data.dtypes
data.isna()
data.describe()
data.corr()
data.columns

Index(['Unnamed: 0', 'users_id', 'Book.Title', 'Book.Author', 'Publisher',
       'ratings[, 3]'],
      dtype='object')

In [3]:
# change column names
book_data= data.rename(columns = {"Col_1":"Mod_col"}) 
book_data = data.rename(columns = {"Book.Title":"book_title"})
book_data.columns

Index(['Unnamed: 0', 'users_id', 'book_title', 'Book.Author', 'Publisher',
       'ratings[, 3]'],
      dtype='object')

In [4]:
book_data = book_data.iloc[:,1:]
book_data.columns

Index(['users_id', 'book_title', 'Book.Author', 'Publisher', 'ratings[, 3]'], dtype='object')

In [5]:

from sklearn.feature_extraction.text import TfidfVectorizer #term frequencey- inverse document frequncy is a numerical statistic that is intended to reflect how important a word is to document in a collecion or corpus

# Creating a Tfidf Vectorizer to remove all stop words
tfidf = TfidfVectorizer(stop_words="english")    #taking stop words from tfid vectorizer

In [6]:
book_data["book_title"].isnull().sum() 


0

In [7]:

tfidf_matrix = tfidf.fit_transform(book_data["book_title"])   #Transform a count matrix to a normalized tf or tf-idf representation
tfidf_matrix.shape #(5000, 10990)

(5000, 10990)

In [8]:
from sklearn.metrics.pairwise import linear_kernel

# Computing the cosine similarity on Tfidf matrix
cosine_sim_matrix = linear_kernel(tfidf_matrix,tfidf_matrix)

In [9]:

# creating a mapping of anime name to index number 
book_index = pd.Series(book_data.index,index=book_data['book_title']).drop_duplicates()



In [10]:
def get_book_recommendations(book_title,topN=10): 
    book_id = book_index[book_title] 
    cosine_scores = list(enumerate(cosine_sim_matrix[book_id])) 
    cosine_scores = sorted(cosine_scores,key=lambda x:x[1],reverse = True)
    cosine_scores_10 = cosine_scores[0:topN+1]
    book_idx  =  [i[0] for i in cosine_scores_10]
    book_scores =  [i[1] for i in cosine_scores_10]
    book_similar_show = pd.DataFrame(columns=["Title","Score"])
    book_similar_show["Title"] = book_data.loc[book_idx,"book_title"]
    book_similar_show["Score"] = book_scores
    book_similar_show.reset_index(inplace=True)  
    book_similar_show.drop(["index"],axis=1,inplace=True)
    print (book_similar_show)

In [11]:
get_book_recommendations("PLEADING GUILTY",topN=15)

                                                Title     Score
0                                     PLEADING GUILTY  1.000000
1                                     Pleading Guilty  1.000000
2                                       Guilty as Sin  0.487075
3   Guilty Pleasures (Anita Blake Vampire Hunter (...  0.270672
4    Whiteside Limited;http://images.amazon.com/im...  0.001974
5                                 Classical Mythology  0.000000
6                                        Clara Callan  0.000000
7                                Decision in Normandy  0.000000
8   Flu: The Story of the Great Influenza Pandemic...  0.000000
9                              The Mummies of Urumchi  0.000000
10                             The Kitchen God's Wife  0.000000
11  What If?: The World's Foremost Military Histor...  0.000000
12  Under the Black Flag: The Romance and the Real...  0.000000
13            Where You'll Find Me: And Other Stories  0.000000
14                        Nights Below S